In [95]:
from imslp import client, interfaces, helpers
from bs4 import BeautifulSoup
from bs4.element import Tag
import requests
import typing
import re

In [2]:
def matches_meta_attribute(header_text):
    for attribute in META_ATTRIBUTES:
        if attribute in header_text or header_text in attribute:
            return attribute
    return None

In [3]:
# Function to check if header text matches any target attribute, accounting for nested span tags
def header_matches_meta_attribute(header):
    # Normalize header text by stripping and joining with space if span-separated
    header_text = ' '.join(header.stripped_strings)
    for attribute in META_ATTRIBUTES:
        if attribute in header_text or header_text in attribute:
            return attribute
    return None

In [86]:
def process_general_info(div: Tag):
    metadata = {}
    
    for row in div.find_all("tr"):
        header_cell = row.find("th")
        value_cell = row.find("td")
        
        if header_cell and value_cell:
            matched_attribute = header_matches_meta_attribute(header_cell)
            
            # If there's a matching attribute, process the value
            if matched_attribute:
                if matched_attribute == "External Links":
                    links = [a['href'] for a in value_cell.find_all("a", href=True)]
                    metadata[matched_attribute] = links
                elif matched_attribute in "Movements/Sections":
                    # print("DATA")
                    # print(value_cell)
                    metadata[matched_attribute] = handle_movements(value_cell)
                elif matched_attribute== 'Year/Date of Composition':
                    year = int(value_cell.get_text()[:4])
                    metadata[matched_attribute] = year
                elif matched_attribute == 'Key':
                    cleansed_key = None
                    # enum map...
                    if len(value_cell.get_text(" ")) < 10: 
                        cleansed_key = value_cell.get_text(" ", strip=True)
                    metadata[matched_attribute] = cleansed_key
                else:
                    # General case for other fields, removing non-breaking spaces
                    clean_text = value_cell.get_text(" ", strip=True).replace('\xa0', ' ')
                    metadata[matched_attribute] = clean_text
    return metadata

In [96]:
def handle_movements(data: Tag):
    movements = []

    # Helper function to extract a leading number if present
    def extract_number(name, default_index):
        match = re.match(r"(\d+)\.", name)  # Look for a leading digit followed by a dot
        return int(match.group(1)) if match else default_index + 1

    # Check for ordered list (<ol>) for movements
    movement_list = data.find('ol')
    if movement_list:
        for index, li in enumerate(movement_list.find_all('li')):
            line = li.get_text(strip=True).replace("\xa0", " ")
            number = extract_number(line, index)  # Extract number or use index
            
            if "(" in line and line.endswith(")"):
                name, key_signature = line.rsplit("(", 1)
                key_signature = key_signature.rstrip(")").replace("\xa0", " ")
                movements.append({
                    "type": "movement",
                    "number": number,
                    "name": name.strip(),
                    "key_signature": key_signature.strip()
                })
            else:
                movements.append({
                    "type": "movement",
                    "number": number,
                    "name": line.strip(),
                    "key_signature": None
                })

    # Check for description list (<dl>) for suite pieces
    piece_list = data.find('dl')
    if piece_list:
        for index, dd in enumerate(piece_list.find_all('dd')):
            line = dd.get_text(strip=True).replace("\xa0", " ")
            number = extract_number(line, index)  # Extract number or use index
            
            if "(" in line and line.endswith(")"):
                name, key_signature = line.rsplit("(", 1)
                key_signature = key_signature.rstrip(")").replace("\xa0", " ")
                movements.append({
                    "type": "piece",
                    "number": number,
                    "name": name.strip(),
                    "key_signature": key_signature.strip()
                })
            else:
                movements.append({
                    "type": "piece",
                    "number": number,
                    "name": line.strip(),
                    "key_signature": None
                })

    return movements
# handle_movements(general_info_div)

In [99]:
results = client.search_works(composer="Rachmaninoff", title="Preludes")

AttributeError: module 'imslp.client' has no attribute 'search_works'

In [108]:
META_ATTRIBUTES = ['Work Title', 'Opus/Catalogue Number', 'Key', 'Year/Date of Composition', 'Piece Style', 'External Links', 'Movements/Section' ]
for index, result in enumerate(results):
    url = result['permlink']
    data = requests.get(url)
    soup = BeautifulSoup(data.content, "html.parser")
    # print(soup.prettify())
    # for file_block in soup.select('.we_file_first, .we_file_hideentry'):
    # # Get the title and download link
    #     title_tag = file_block.select_one('.we_file_download a.external.text')
    #     print(title_tag)
    #     title = title_tag.text.strip() if title_tag else "N/A"
    #     download_link = title_tag['href'] if title_tag else "N/A"
    #     print(download_link)
        
        # Get file information (size and pages)

            
    # print(f"Title: {title}")
    # print(f"Download Link: {download_link}")

    # print("-" * 40)
    # Find the div that contains the metadata table
    # general_info_div = soup.find("div", class_="wi_body")
    
    # # # print(general_info_div)
    # print(process_general_info(general_info_div))
    # break
    break
    # break  # Remove this if you want to process more than just the second item
    # il mio primo chopin?